In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import SCM_MEBM as scm_mebm
import matplotlib as mpl

In [ ]:
mpl.rcParams['axes.titlesize']        = 14 
mpl.rcParams['legend.fontsize']       = 14  
mpl.rcParams['legend.title_fontsize'] = 14  
mpl.rcParams['xtick.labelsize']       = 14  
mpl.rcParams['ytick.labelsize']       = 14  
mpl.rcParams['axes.labelsize']        = 14 

In [ ]:
# --- set up grid and time-stepping ---

n   = 120                           # number of grid points 
dx  = 2./n                          # width of grid box
x   = np.linspace(-1+dx/2,1-dx/2,n) # native grid
xb  = np.linspace(-1+dx,1-dx,n-1)   # boundaries of grid
nt  = 1000                          # number of timesteps
dur = 100                           # length of simulation in years
dt  = 1./nt                         # timestep in years
Rf  = 0                             # radiative forcing value

grid = {'n': n, 'dx': dx, 'x': x, 'xb': xb, 'nt': nt, 'dur': dur, 'dt': dt} 

Ti  = 7.5+20*(1-2*x**2)             # initial condition 

# --- run the SCM-MEBM ---

time, Ts_ctrl, Hi_ctrl, Sw_ctrl = scm_mebm.model(grid, Ti, F = Rf, sea_ice_albedo = 'on', sea_ice_thermodynamics = 'on')

In [ ]:
# --- process model output ---

# extract Northern Hemisphere output
n_2   = int(n/2)+1                 # number of grid points in the Northern Hemisphere
x_n   = x[-n_2:]                   # sine of latitude
lat_n = np.rad2deg(np.arcsin(x_n)) # latitude
Ts_ctrl = Ts_ctrl[-n_2:,:]         # surface temperature 
Hi_ctrl = Hi_ctrl[-n_2:,:]         # sea ice thickness

## Make some plots of the control experiment

In [ ]:
# convert time from idealized model into months for plotting [ assume each month has 30 days ]
xticks = np.arange(15,360,30)/360
xtick_labels = np.array(['J','F','M','A','M','J','J','A','S','O','N','D'])

# --- plot the seasonal data for the whole Northern Hemisphere ---

fig = plt.figure(figsize=(12,8))

# plot seasonal surface temperature
plt.subplot(221)
clevsh = np.linspace(-20,20,11)
plt.contourf(time,lat_n,Ts_ctrl,clevsh,cmap='viridis',extend='both')
cb = plt.colorbar()
cb.set_label('Surface temperature [ °C ]')
plt.xticks(xticks,xtick_labels)
plt.xlabel('Month')
plt.ylabel('Latitude')
plt.ylim(0,np.max(lat_n))
plt.yticks([0,25,50,75],['0°','25°N','50°N','75°N'])

# plot seasonal sea ice thickness
plt.subplot(222)
clevsh = np.linspace(0,2,11)
plt.contourf(time,lat_n,Hi_ctrl,clevsh,extend='max',cmap='Blues_r')
cb = plt.colorbar()
cb.set_label('Sea ice thickness [ m ]')
# plot the ice edge
plt.contour(time,lat_n,Hi_ctrl,[0],colors='C3')
plt.xticks(xticks,xtick_labels)
plt.xlabel('Month')
plt.ylabel('Latitude')
plt.ylim(0,np.max(lat_n))
plt.yticks([0,25,50,75],['0°','25°N','50°N','75°N'])

# plot surface temperature profiles - averaged by latitude
plt.subplot(223)
Annual, = plt.plot(lat_n,np.nanmean(Ts_ctrl[:,:],axis=1),'k',label='Annual',lw=2)

# 500:750 corresponds to June, July, August
Summer, = plt.plot(lat_n,np.nanmean(Ts_ctrl[:,500:750],axis=1),'k--',label='Summer',lw=2)

# 0:250 corresponds to January, February, March
Winter, = plt.plot(lat_n,np.nanmean(Ts_ctrl[:,0:250],axis=1),'k:',label='Winter',lw=2)

plt.xticks([0,25,50,75],['0°','25°N','50°N','75°N'])
plt.xlim(0,np.max(lat_n))
plt.ylim(-30,30)
plt.yticks([-30,-15,0,15,30])
plt.xlabel('Latitude')
plt.ylabel('Surface temperature [ °C ]')
plt.legend(handles = [Annual,Summer,Winter],loc=0)
plt.grid()

# plot sea ice thickness profiles - averaged by latitude
plt.subplot(224)
plt.plot(lat_n,np.nanmean(Hi_ctrl[:,:],axis=1),'k',label='Annual',lw=2)

# 500:750 corresponds to June, July, August
plt.plot(lat_n,np.nanmean(Hi_ctrl[:,500:750],axis=1),'k--',lw=2)

# 0:250 corresponds to January, February, March
plt.plot(lat_n,np.nanmean(Hi_ctrl[:,0:250],axis=1),'k:',lw=2)

plt.ylim([0,4])
plt.yticks([0,1,2,3,4])
plt.xticks([45,60,75,90],['45°N','60°N','75°N','90°N'])
plt.xlim(45,np.max(lat_n))
plt.xlabel('Latitude')
plt.ylabel('Sea ice thickness [ m ]')
plt.grid()

plt.rcParams.update({'font.size': 14})

plt.tight_layout() 
plt.savefig('figures/fig1.pdf',bbox_inches='tight')

## Model output at Utqiagvik, Alaska

In [ ]:
# print data for Utqiagvik, Alaska (71.3°N) around April 7-11. This corresponds to t = 0.27 and x = 0.95 or lat = 71.3
print("--- April 7-11 in Utqiagvik, Alaska --- \n")

value_time = 0.27  # Time of year that in the idealized model corresponds to April 7-11
value_loc  = 71.3  # Latitude of Utqiagvik, Alaska

# get index for this date
ind_time = np.argmin(np.abs(time - value_time))
# get index for this location
ind_loc  = np.argmin(np.abs(lat_n - value_loc))

# print surface temperature and sea ice thickness values
print("Surface temperature: " + str(np.round(Ts_ctrl[ind_loc,ind_time],2))+" °C \n")
print("Sea ice thickness: "   + str(np.round(Hi_ctrl[ind_loc,ind_time],2))+" m")

In [ ]:
# convert time into months for plotting [ assume each month has 30 days ]
xticks = np.arange(15,360,30)/360
xtick_labels = np.array(['J','F','M','A','M','J','J','A','S','O','N','D'])

fig = plt.figure(figsize=(12,6))

# plot the seasonal cycle of surface temperature in Utqiagvik, AK
plt.subplot(121)
plt.plot(time,Ts_ctrl[ind_loc,:],lw=2)
plt.plot(time[ind_time],Ts_ctrl[ind_loc,ind_time],'o',markeredgecolor='white',markersize=10,markerfacecolor='C3')
plt.xticks(xticks,xtick_labels)
plt.xlabel('Month')
plt.ylim(-27,12)
plt.yticks(np.arange(-24,10,3))
plt.ylabel('Surface temperature [ °C ]')
plt.grid()

# plot the seasonal cycle of sea ice thickness in Utqiagvik, AK
plt.subplot(122)
plt.plot(time,Hi_ctrl[ind_loc,:],lw=2)
plt.plot(time[ind_time],Hi_ctrl[ind_loc,ind_time],'o',markeredgecolor='white',markersize=10,markerfacecolor='C3')
plt.xticks(xticks,xtick_labels)
plt.xlabel('Month')
plt.ylim(-0.2,2.4)
plt.yticks(np.arange(0,2.4,0.2))
plt.ylabel('Sea ice thickness [ m ]')
plt.grid()

plt.suptitle('Utqiagvik, Alaska (71.3°N)',fontsize=14)

plt.rcParams.update({'font.size': 14})

plt.tight_layout() 
plt.savefig('figures/fig2.pdf',bbox_inches='tight')

In [ ]:
# --- Enter YOUR code for Questions 3-7 ---

In [ ]:
# --- Enter YOUR code for Questions 3-7 ---

In [ ]:
# --- Enter YOUR code for Questions 3-7 ---